In [ ]:
#installing streamlit and pyngrok
!pip -q install streamlit
!pip -q install pyngrok
!pip install tensorflow-io
!pip install tensorflow-addons

In [ ]:
from pyngrok import ngrok

In [ ]:
%%writefile app.py

# Importing the necessary libraries
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_addons as tfa
import pandas as pd  
import numpy as np
import os
import tensorflow_hub as hub
import datetime
import warnings
warnings.filterwarnings('ignore')
from pyngrok import ngrok
import streamlit as st

#giving a title to webpage
st.title("Species Audio Detection")

id = st.number_input("Enter the index of the file")


def load_files():
    #https://machinelearningmastery.com/save-load-keras-deep-learning-models/


    #Load the models from json files
    #load the weights
    json_file_1=open('/content/drive/MyDrive/Appliedai files/Case_Study_2/model_1.json','r')
    loaded_model_json_1=json_file_1.read()
    json_file_1.close()
    loaded_model_1= tf.keras.models.model_from_json(loaded_model_json_1)
    loaded_model_1.load_weights('/content/drive/MyDrive/Appliedai files/Case_Study_2/CS2_modelling2/model_6/weights-26-0.6329.hdf5')

    json_file_2=open('/content/drive/MyDrive/Appliedai files/Case_Study_2/model_2.json','r')
    loaded_model_json_2=json_file_2.read()
    json_file_2.close()
    loaded_model_2= tf.keras.models.model_from_json(loaded_model_json_2)
    loaded_model_2.load_weights('/content/drive/MyDrive/Appliedai files/Case_Study_2/CS2_modelling2/model_1/21-0.9206-0.7486.h5')

    json_file_3=open('/content/drive/MyDrive/Appliedai files/Case_Study_2/model_3.json','r')
    loaded_model_json_3=json_file_3.read()
    json_file_3.close()
    loaded_model_3= tf.keras.models.model_from_json(loaded_model_json_3)
    loaded_model_3.load_weights('/content/drive/MyDrive/Appliedai files/Case_Study_2/CS2_modelling2/model_2/32-0.8647-0.7383.h5')

    json_file_4=open('/content/drive/MyDrive/Appliedai files/Case_Study_2/model_4.json','r')
    loaded_model_json_4=json_file_4.read()
    json_file_4.close()
    loaded_model_4= tf.keras.models.model_from_json(loaded_model_json_4)
    loaded_model_4.load_weights('/content/drive/MyDrive/Appliedai files/Case_Study_2/model_1/weights-69-0.6032.hdf5')

    return loaded_model_1,loaded_model_2,loaded_model_3,loaded_model_4
 
def data_preprocess(id):
    #load Yamnet and get embeddings
    yamnet_model_handle= 'https://tfhub.dev/google/yamnet/1'
    yamnet_model=hub.load(yamnet_model_handle)
    data=pd.read_csv('/content/drive/MyDrive/Appliedai files/Case_Study_2/train_tp.csv')
    #loading the raw audio
    audio= tfio.IOTensor.graph(tf.int16).from_audio('/content/drive/MyDrive/Appliedai files/Case_Study_2/train/'+data['recording_id'][id]+'.flac')
    audio_slice = audio[100:]

    #remove last dimension
    audio_tensor= tf.squeeze(audio_slice,axis=[-1])
    audio_tensor= tf.cast(audio_tensor, tf.float32)

    #convert to spectrogram
    spectrogram = tfio.audio.spectrogram(audio_tensor,nfft=2048,window=2048,stride=512)

    #convert to melscale
    mel_spectrogram=tfio.audio.melscale(spectrogram,rate=48000,mels=384,fmin=40,fmax=24000)

    #convert to dbscale
    mel_spectrogram = tfio.audio.dbscale(mel_spectrogram, top_db=80)

    #Expanding the dimensions
    image=tf.expand_dims(mel_spectrogram,axis=-1)
    #Resizing
    image=tf.image.resize(image,[384,768])
    #converting to rgb image
    image=tf.image.grayscale_to_rgb(image)

    #resample the audio to feed the signal to yamnet model
    sample_rate = tf.cast(48000, dtype=tf.int64)
    audio_tensor = tfio.audio.resample(audio_tensor, rate_in=sample_rate, rate_out=16000)

    #Get the embeddings from yamnet
    scores, embeddings, spectrogram = yamnet_model(audio_tensor)

    return image,embeddings
  
def predict(id):

    start_time=datetime.datetime.now()
    #load files
    loaded_model_1,loaded_model_2,loaded_model_3,loaded_model_4=load_files()

    #preprocess audio data
    image,embeddings= data_preprocess(id)
    image=tf.expand_dims(image,axis=0)
    image=tf.reshape(image,[1,384,768,3])
    embeddings=tf.expand_dims(embeddings,axis=0)
    embeddings=tf.reshape(embeddings,[1,124,1024])
    
    cnn_models=[loaded_model_1,loaded_model_2,loaded_model_3]
    #rec_id= data['recording'][id]
    #Ensembling the models and making predictions 
    y_pred = []
    final_pred=[]
    for model in cnn_models:
      pred = tf.sigmoid(model(image)).numpy()
      y_pred.append(pred)

    y_pred=np.asarray(y_pred)
    cnn_mean_pred=np.mean(y_pred,axis=0)
    rnn_pred=loaded_model_4.predict(embeddings)
    cnn_mean_pred=list(cnn_mean_pred)
    rnn_pred=list(rnn_pred)

    final_pred.append(cnn_mean_pred)
    final_pred.append(rnn_pred)
    final_pred=np.asarray(final_pred)
    mean_final_pred=np.mean(final_pred,axis=0)

    #Return top three predictions
    predictions=np.argsort(mean_final_pred,axis=None)[::-1]
    top_3_predictions= predictions[0:3]

    end_time=datetime.datetime.now()-start_time
    #print("Total time taken for prediction:",datetime.datetime.now()-start_time)
    return top_3_predictions,end_time

top_pred,end_time = predict(id)


#display the predicted output,recording id and time taken in webpage
st.write('Top 3 predicted species in the audio are:, '+str(top_pred[0])+', '+str(top_pred[1])+','+str(top_pred[2]))

st.write("Time taken:", end_time)



Overwriting app.py


In [ ]:
#authentiacting tunnel
!ngrok authtoken 1s9EWYEkYGU9J6chwq2Z0Hucw5R_2iBbA53YyhmegQY3b67nm

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

NgrokTunnel: "http://807f51717b1f.ngrok.io" -> "http://localhost:80"
2021-08-02 09:40:24.842323: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-02 09:40:26.505138: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-02 09:40:26.523545: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-02 09:40:26.524156: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-08-02 09:40:26.524223: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudar

### Video for deployement

https://youtu.be/hHQnL6YGZeU